# Wave Phenomena
Sean Wade

In [18]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation
from scipy.integrate import odeint
from scipy import optimize as opt
from __future__ import division

## Problem 1

In [2]:
def solve(J, M, t, f, g, a, b, s, iters=None):
    
    x = np.linspace(a, b, J+1)
    if iters == None:
        iters = M - 1
        
    del_x = float(b-a) / J
    del_t = t / M
    
    lmbda = (s * del_t) / del_x
    U_0 = np.array([f(k) for k in x])[1:-1]
    n = len(U_0)
    
    A = np.diag(2 * np.ones(n))
    A -= np.diag(2 * (lmbda**2) * np.ones(n))
    A += np.diag(np.ones(n-1)*lmbda**2, k=-1)
    A += np.diag(np.ones(n-1)*lmbda**2, k=1)
    
    U_1 = U_0 + g(x[1:-1]) * del_t+((lmbda**2)/2.) * ((np.array([f(i) for i in x])[0:-2])-2*U_0+np.array([f(j) for j in x])[2:])
    U_M = U_1
    U_M_minus = U_0
    U_M_1 = U_1
    
    for i in range(iters):
        U_M_1 = np.dot(A, U_M) - U_M_minus
        U_M_minus = U_M
        U_M = U_M_1
        
    U_M_1 = np.append(f(a), U_M_1)
    U_M_1 = np.append(U_M_1, f(b))
    return U_M_1

In [3]:
def real_solution(x,t):
    return np.sin(2*np.pi*x) * np.cos(2*np.pi*t)

In [4]:
lmbda = .2
J, M = 5, 5
a, b = 0, 1
t = .5
s = 1
f = lambda x: np.sin(2*np.pi*x)
g = lambda x: 0.0 * x

print solve(J, M, t, f, g, a, b, s, iters=None)
print "-"*30
x = np.linspace(0,1,6)
print real_solution(x,.5)

[  0.00000000e+00  -9.39116409e-01  -5.80405860e-01   5.80405860e-01
   9.39116409e-01  -2.44929360e-16]
------------------------------
[ -0.00000000e+00  -9.51056516e-01  -5.87785252e-01   5.87785252e-01
   9.51056516e-01   2.44929360e-16]


## Problem 2

In [20]:
def anim(J, M, t, f, g, a, b, s): 
    x = np.linspace(a, b, J+1)
    y = np.array([f(k) for k in x])
        
    my_fig = plt.figure()
    plt.axes(xlim=(a,b), ylim=(-t,t))
    line, = plt.plot([], [])
    
    def animate(i):
        line.set_data(x, solve(J, M, t, f, g, a, b, s, iters=i))
        return line,
    
    an = animation.FuncAnimation(my_fig, animate, frames=y.size, interval=20)
    plt.show()

In [6]:
J = 200
t = 1.
s = 1.
a = 0
b = 1.
m = 20

f = lambda x: .2 * np.exp(-1*m**2*(x-.5)**2)
g = lambda x: -.4 * m**2 * (x-.5) * np.exp((-1*m**2)*(x-.5)**2)

In [7]:
anim(J, 220, t, f, g, a, b, s)
anim(J, 180, t, f, g, a, b, s)

## Problem 3

In [8]:
m = 20
f = lambda x: .2*np.exp(-1*m**2*(x-.5)**2)
g = lambda x: 0.0 * m

In [9]:
anim(200, 440, 2, f, g, 0, 1, 1)

## Problem 4

In [10]:
def f(x):
    if (5/11 < x) and x < (6/11):
        return 1/3
    else:
        return 0

g = lambda x: 0

In [11]:
anim(200, 440, 2, f, g, 0, 1, 1)

## Problem 5

In [44]:
def crank_nicol(f, J, M, xa, xb, ta, tb, nu, iters=M-1):
    del_t = float(tb-ta) / M
    del_x = float(xb-xa) / J
    
    k_1 = del_t / (4*del_x)
    k_2 = del_t / (2*del_x**2)
    
    x_val = np.linspace(xa,xb,J+1)
    U_0 = f(x_val)
    
    def new_U_maker(x):
        ph = np.zeros_like(U_0)
        ph[1:-1]= x[1:-1] - U_0[1:-1] - k_1*((s-x[1:-1])*(x[2:]-x[:-2]) + \
        (s-U_0[1:-1])*(U_0[2:] - U_0[:-2])) - k_2*((x[2:]-2*x[1:-1]+x[:-2])+\
        (U_0[2:]-2*U_0[1:-1]+U_0[:-2]))
        
        ph[0] = x[0]-U_0[0]
        ph[-1] = x[-1]-U_0[-1]
        
        return ph
    
    for i in xrange(iters):
        U_0 = opt.fsolve(new_U_maker, np.zeros(J+1))
        
    return U_0

In [45]:
def crank_animation(J, M, f, xa, xb, ta, tb):
    x = np.linspace(xa, xb, J+1)
    y = np.array([f(k) for k in x])
    an_fig = plt.figure()
    plt.axes(xlim=(xa,xb), ylim=(0,6))
    line, = plt.plot([], [])
    
    def animate(i):
        line.set_data(x, crank_nicol(f, J, M, xa, xb, ta, tb, nu, iters=i))
        return line,
  
    ans = animation.FuncAnimation(an_fig, animate, frames=y.size, interval=20)
    plt.show()

In [46]:
delta = 0.
u_minus = 5.
u_plus = 1.
a = (u_minus - u_plus)/2.
s = (u_minus+u_plus)/2.
nu = 1.
xa, xb = -20, 20
ta, tb = 0., 1.0
J = 150
M = 350

u_hat = lambda x: s-a * np.tanh(a*x/float(2*nu) + delta)
v = lambda x: 3.5 * (np.sin(3*x)+1) * (1/np.sqrt(2*np.pi)) * np.exp(-x**2/2)
f = lambda x: u_hat(x) + v(x)

In [47]:
crank_animation(J, M, f, xa, xb, ta, tb)